In [ ]:
# Cell 1: Setup
import sys
import os
from pathlib import Path
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import yaml

# Add scripts directory to path
repo_root = Path('.').resolve()
scripts_path = repo_root / 'scripts'
if str(scripts_path) not in sys.path:
    sys.path.append(str(scripts_path))

# Import project classes
from utils.config_loader import load_config
from inventory import Inventory
from bom import BoM
from health_calculator import HealthCalculator
from check_assets import VerificationEngine, VerificationReport
from utils.api_clients import create_llm_client, create_footprint_client
from utils.kicad_utils import LibraryManager # Pass config, project context?
from utils.file_utils import DatasheetManager # Pass project context?
# from scripts.render_footprint import FootprintRenderer # If using LMM check
from scripts.integrate_assets import promote_reviewed_asset, update_bom_footprint # Helpers
from scripts.doc_generator import DocumentationGenerator
# from scripts.populate_kicad_fields import PopulateKiCadFields # Or directly generate here
# from scripts.calculate_fp_area import FootprintAreaCalculator # Or directly use lib_manager

# --- User Input ---
PROJECT_NAME = "GreenhouseController" # MUST MATCH the KiCad project folder name
KICAD_BOM_EXPORT_FILENAME = "GreenhouseController-bom.csv" # CHANGE to your exported BoM filename (CSV or XML)
# --- End User Input ---

project_dir = repo_root / PROJECT_NAME
kicad_export_path = project_dir / KICAD_BOM_EXPORT_FILENAME
canonical_bom_path = project_dir / "bom.yaml" # Where the final verified BoM lives

# Load Config and Initialize
try:
    config_obj = load_config()
    inventory = Inventory(repo_root / "inventory.yaml")
    inventory.load()
    llm = create_llm_client(config_obj)
    fp_api = create_footprint_client(config_obj)
    # These need project context / config for paths
    lib_manager = LibraryManager(config_obj, project_dir)
    ds_manager = DatasheetManager(project_dir)
    health_calc = HealthCalculator(config_obj.health_rules)
    # fp_renderer = FootprintRenderer(...) # Initialize if needed
    verifier = VerificationEngine(config_obj, inventory, lib_manager, ds_manager, llm, fp_api, health_calc) #, fp_renderer)
    project_bom = BoM(canonical_bom_path) # Manages the canonical YAML
    print("Setup complete.")
except Exception as e:
    print(f"Error during setup: {e}")
    raise # Stop if setup fails






In [ ]:
# Cell 2: Load BoM from KiCad Export
print(f"Loading BoM from KiCad export: {kicad_export_path}")
loaded_ok = project_bom.load_from_kicad_export(kicad_export_path)
if loaded_ok:
    display(project_bom.to_dataframe(include_health=False).head()) # Show first few rows
    # Immediately save this initial load to the canonical path for the verifier to work on
    project_bom.save()
else:
    print("Stopping due to BoM load error.")
    # Stop notebook execution?

In [ ]:
# Cell 3: Asset Verification Loop
verify_llm_docs = widgets.Checkbox(value=False, description='Perform LLM Doc Checks (Experimental)')
verify_lmm_fp = widgets.Checkbox(value=False, description='Perform LMM Footprint Checks (Experimental)')
run_verify_button = widgets.Button(description="Run Verification Checks")
process_accepted_button = widgets.Button(description="Process Accepted Assets & Re-Verify", disabled=True) # Disabled initially
verification_output = widgets.Output()
acceptance_widgets = {} # Store checkboxes for accepting reviewed items: {ref: {'fp': Checkbox, 'sym': Checkbox}}

def run_verification(b=None):
    global acceptance_widgets # Allow modification
    with verification_output:
        verification_output.clear_output(wait=True) # Clear previous output but wait for new
        print("Running verification... This may take time due to API calls.")
        # Reload the canonical BoM in case it was updated externally? Or assume object state is master?
        project_bom.load_canonical() # Load latest state before verifying

        report = verifier.verify_bom(
            project_bom,
            check_llm_doc=verify_llm_docs.value,
            # check_lmm_fp=verify_lmm_fp.value # If implemented
        )

        # Display report DataFrame including health scores
        bom_df = project_bom.to_dataframe(include_health=True)
        display(bom_df)
        print("\n--- Verification Summary ---")
        print(report.model_dump_json(indent=2))

        # Create acceptance widgets for items pending review
        acceptance_widgets = {}
        review_widgets_list = []
        print("\n--- Assets Pending Review ---")
        needs_processing = False
        for comp in project_bom.get_all_components():
            if comp.status.footprint_verified == 'review_pending':
                 fp_cb = widgets.Checkbox(description=f"Accept Footprint for {comp.ref}", value=False, indent=False)
                 if comp.ref not in acceptance_widgets: acceptance_widgets[comp.ref] = {}
                 acceptance_widgets[comp.ref]['fp'] = fp_cb
                 review_widgets_list.append(fp_cb)
                 needs_processing = True
            # Add similar logic for symbols if API download for symbols is implemented
            # if comp.status.symbol_verified == 'review_pending':
            #      sym_cb = widgets.Checkbox(...)
            #      acceptance_widgets[comp.ref]['sym'] = sym_cb
            #      review_widgets_list.append(sym_cb)
            #      needs_processing = True

        if review_widgets_list:
             display(widgets.VBox(review_widgets_list))
             process_accepted_button.disabled = False
        else:
             print("No assets currently pending review.")
             process_accepted_button.disabled = True

        # Save the updated BoM state after verification completes
        project_bom.save()

def process_accepted(b=None):
     print("\nProcessing accepted assets...")
     processed_something = False
     # Reload just before processing
     project_bom.load_canonical()

     for ref, widgets_dict in acceptance_widgets.items():
          component = project_bom.get_component(ref)
          if not component: continue

          if 'fp' in widgets_dict and widgets_dict['fp'].value:
               print(f" Promoting footprint for {ref}...")
               # Need to get the actual downloaded footprint path/name
               # This might require storing more info during download or finding it in review dir
               # Assuming a naming convention for simplicity:
               assumed_review_fp_name = f"{component.mpn or ref}_footprint.kicad_mod"
               review_dir = repo_root / "libs" / "review" / (component.mpn or ref)
               if promote_reviewed_asset(ref, component.mpn, 'footprint', review_dir, repo_root / "libs"):
                   # Update BoM object - footprint name might change if Lib changes to project lib!
                   # Assuming footprint name stays same but Library changes to project lib name
                   proj_lib_name = (repo_root / "libs" / "footprints.pretty").parent.name # Or get from config
                   new_fp_ref = f"{proj_lib_name}:{Path(assumed_review_fp_name).stem}"
                   update_bom_footprint(project_bom, ref, new_fp_ref) # Updates object in memory
                   component.update_status({'footprint_verified': True, 'api_downloaded': False}) # Update status
                   component.source_info = f"API Verified ({component.source_info})" # Update source
                   processed_something = True
               else:
                   print(f"  Failed to promote footprint for {ref}.")
               widgets_dict['fp'].value = False # Reset checkbox

          # Add similar logic for symbols if implemented

     if processed_something:
          print("Saving updated BoM...")
          project_bom.save() # Save changes to canonical bom.yaml
          print("Re-running verification...")
          run_verification() # Trigger re-verification automatically
     else:
          print("No assets were selected for processing.")

run_verify_button.on_click(run_verification)
process_accepted_button.on_click(process_accepted)

display(widgets.HBox([verify_llm_docs, verify_lmm_fp]))
display(run_verify_button)
display(verification_output) # Display results here
display(process_accepted_button)

In [ ]:
# Cell 4: Generate Documentation
doc_gen_button = widgets.Button(description="Generate Component Review Doc")
doc_gen_output = widgets.Output()

def on_doc_gen_click(b):
     with doc_gen_output:
          doc_gen_output.clear_output()
          print("Generating documentation...")
          try:
               # Reload final verified BoM state
               project_bom.load_canonical()
               # Need renderer initialized if images are desired
               # fp_renderer = FootprintRenderer(...)
               doc_generator = DocumentationGenerator(project_bom, inventory, project_dir.parent) # Pass repo root?
               md_path = project_dir.parent / "docs" / "Component_Review.md"
               doc_generator.generate_report(md_path)
               display(Markdown(f"Documentation generated: [{md_path.relative_to(repo_root)}]({md_path.relative_to(repo_root)})"))
          except Exception as e:
               print(f"Error generating documentation: {e}")

doc_gen_button.on_click(on_doc_gen_click)
display(doc_gen_button, doc_gen_output)

In [ ]:

# Cell 5: Final Checks (Area) & Handoff Prep
final_checks_button = widgets.Button(description="Run Final Checks & Prep")
final_checks_output = widgets.Output()

def on_final_checks_click(b):
     with final_checks_output:
          final_checks_output.clear_output()
          print("Running final checks and generating KiCad data...")
          try:
               project_bom.load_canonical() # Ensure latest BoM

               # 1. Area Calculation (using kicad_utils directly or a dedicated class)
               total_area = 0
               # area_calc = FootprintAreaCalculator(lib_manager)
               # total_area, _ = area_calc.calculate_bom_area(project_bom)
               print(f"Estimated Footprint Area: {total_area:.2f} mm^2 (Needs implementation in kicad_utils)")

               # 2. Power Netlist (Optional)
               # netlist_gen = NetlistGenerator()
               # netlist_path = project_dir / f"{PROJECT_NAME}_power.net"
               # netlist_gen.generate_kicad_power_netlist(project_bom, ..., netlist_path)
               # print(f"Power Netlist generated: {netlist_path.relative_to(repo_root)}")

               # 3. Field Population Data
               # populator = PopulateKiCadFields(project_bom)
               # field_data = populator.generate_clipboard_data()
               field_data = "# Field data generation needs implementation in populate_kicad_fields.py"
               print("\n--- Data for KiCad 'Edit Symbol Fields' ---")
               print(field_data)
               print("------------------------------------------")

          except Exception as e:
               print(f"Error during final checks: {e}")

final_checks_button.on_click(on_final_checks_click)
display(final_checks_button, final_checks_output)


In [ ]:
# Cell 6: Handoff Instructions
display(Markdown(f"""
## Handoff to KiCad

1.  **Review:** Carefully review the generated `docs/Component_Review.md`. Resolve any outstanding issues manually in KiCad libs or `bom.yaml`.
2.  **Open Project:** Open `{PROJECT_NAME}/{PROJECT_NAME}.kicad_pro` in KiCad.
3.  **Schematic:**
    *   Open Eeschema. Ensure all symbols from `bom.yaml` are placed.
    *   Use `Tools -> Edit Symbol Fields` and paste the data generated above to populate Value, Footprint, etc. **Verify Footprint fields!**
    *   (Optional) Import Power Netlist: `File -> Import -> Netlist...` select `{PROJECT_NAME}_power.net`.
    *   Wire the schematic completely.
    *   Run ERC and fix all errors/warnings.
    *   Save schematic and commit `.kicad_sch`.
4.  **Layout:**
    *   Open Pcbnew.
    *   Update PCB from Schematic (`Tools -> Update PCB from Schematic...` or F8). Fix any footprint errors.
    *   Perform component placement.
    *   Route traces. Add copper fills. Define board outline.
    *   Run DRC and fix all errors.
    *   Save PCB and commit `.kicad_pcb`.
5.  **Fabrication:** Generate Gerbers and Drill files from Pcbnew.
"""))